# Ingest ephys session path on bl_new_acquisition.acquisition_sessions table

In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


## Connection to DB

In [2]:
import datajoint as dj
import pandas as pd
import utility.path_utility as pu
import os

bdata          = dj.create_virtual_module('bdata', 'bl_bdata')
shadow_acquisition = dj.create_virtual_module('shadow_acquisition', 'bl_shadow_acquisition')
new_acquisition = dj.create_virtual_module('new_acquisition', 'bl_new_acquisition')
new_lab = dj.create_virtual_module('new_lav', 'bl_new_lab')
ratinfo        = dj.create_virtual_module('ratinfo', 'bl_ratinfo')

Connecting alvaros@datajoint01.pni.princeton.edu:3306


## Get all sessions info

In [74]:
list_sessions_fields = ['sessid', 'session_rat', 'session_userid', 'session_rigid', 'session_date']
sessions_df = pd.DataFrame(new_acquisition.Sessions.fetch(*list_sessions_fields, order_by='sessid desc', as_dict=True))
list_contacts_fields = ['user_id', 'experimenter']
contact_df = pd.DataFrame(new_lab.Contacts.fetch(*list_contacts_fields, as_dict=True))
sessions_df = sessions_df.merge(contact_df, left_on='session_userid', right_on='user_id')
sessions_df['session_date'] = sessions_df['session_date'].astype('str')

# 1. Raw session processing

## 1.1 Construct and find nominal paths

In [75]:
ephys_root = dj.config['custom']['ephys_root_data_dir']
sessions_df['subject_path'] = sessions_df.apply(lambda x: pu.combine_str_path(ephys_root, [x['experimenter'], x['session_rat']]), axis=1)
sessions_df['nominal_session_path'] = sessions_df.apply(lambda x: pu.check_date_directory(x['subject_path'], x['session_date']), axis=1)

## 1.2 Filter only sessions with nominal path found

In [76]:
raw_sessions_df_nom_path_found = sessions_df.loc[~sessions_df['nominal_session_path'].isin(pu.path_not_found_dict.values()), :]
raw_sessions_df_nom_path_found = raw_sessions_df_nom_path_found.reset_index(drop=True)

## 1.3 Enumerate all possible directories for each session

In [77]:
#If multiple paths found, this will create a record for each "possibility"
raw_sessions_df_nom_path_found = raw_sessions_df_nom_path_found.explode(['nominal_session_path'])
raw_sessions_df_nom_path_found = raw_sessions_df_nom_path_found.sort_values(by=['sessid'])

## 1.4 Find session files in nominal directories and childs

In [78]:
raw_sessions_df_nom_path_found['real_raw_session_path'] = \
raw_sessions_df_nom_path_found.apply(lambda x: pu.find_file_pattern_dir(x['nominal_session_path'], \
                                                                         pu.file_pattern_ephys_session['raw_np_files']),axis=1)

#If several recoring files are found inside a "parent" path
raw_sessions_df_nom_path_found = raw_sessions_df_nom_path_found.explode(['real_raw_session_path'])

raw_sessions_df_found = raw_sessions_df_nom_path_found.loc[~raw_sessions_df_nom_path_found['real_raw_session_path'].isin(pu.path_not_found_dict.values()), :]
raw_sessions_df_found['raw_session_rel_path'] = raw_sessions_df_found.loc[:,'real_raw_session_path'].str.replace(dj.config['custom']['ephys_root_data_dir'], '', regex=False)
raw_sessions_df_found = raw_sessions_df_found.reset_index(drop=True)

c:\users\stephan\anaconda3\envs\brody_ephys_elements_two\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [79]:
raw_sessions_df_found

,sessid,session_rat,session_userid,session_rigid,session_date,user_id,experimenter,subject_path,nominal_session_path,real_raw_session_path,raw_session_rel_path
0,585444,T170,zhihaol,202,2018-03-16,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T170/T170_2018_03_16
1,585488,T173,zhihaol,202,2018-03-16,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T173/T173_bank1_2018_03_16
2,586335,T170,zhihaol,202,2018-03-19,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T170/T170_2018_03_19
3,586376,T173,zhihaol,202,2018-03-19,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T173/T173_bank0_2018_03_19
4,588819,T173,zhihaol,202,2018-03-29,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T173/T173_bank2_2018_03_29
...,...,...,...,...,...,...,...,...,...,...,...
733,798430,T274,zhihaol,202,2020-12-13,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_13c_g0
734,798586,T274,zhihaol,202,2020-12-14,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_14_g0
735,798742,T274,zhihaol,202,2020-12-15,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_15_g0
736,798897,T274,zhihaol,202,2020-12-16,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_16_g0


# 2. Sorted session processing

## 2.1 Construct and find nominal paths

In [80]:
cluster_root = dj.config['custom']['clustering_root_data_dir']
raw_sessions_df_found['subject_cluster_path'] = raw_sessions_df_found.apply(lambda x: pu.combine_str_path(cluster_root, [x['experimenter'], x['session_rat']]), axis=1)
raw_sessions_df_found['nominal_cluster_session_path'] = raw_sessions_df_found.apply(lambda x: pu.check_date_directory(x['subject_cluster_path'], x['session_date']), axis=1)


## 2.2 Filter only sessions with nominal path found

In [81]:
cluster_sessions_df_nom_path_found = raw_sessions_df_found.loc[~raw_sessions_df_found['nominal_cluster_session_path'].isin(pu.path_not_found_dict.values()), :]
cluster_sessions_df_nom_path_found = cluster_sessions_df_nom_path_found.reset_index(drop=True)


## 2.3 Enumerate all possible directories for each session

In [82]:
#If multiple paths found, this will create a record for each "possibility"
cluster_sessions_df_nom_path_found = cluster_sessions_df_nom_path_found.explode(['nominal_cluster_session_path'])
cluster_sessions_df_nom_path_found = cluster_sessions_df_nom_path_found.sort_values(by=['sessid'])


## 2.4 Find session files in nominal directories and childs

In [83]:
cluster_sessions_df_nom_path_found['real_cluster_session_path'] =\
cluster_sessions_df_nom_path_found.apply(lambda x: pu.find_file_pattern_dir(x['nominal_cluster_session_path'],\
                                                                     pu.file_pattern_ephys_session['sorted_np_files']),axis=1)

#If several recoring files are found inside a "parent" path
cluster_sessions_df_nom_path_found = cluster_sessions_df_nom_path_found.explode(['real_cluster_session_path'])


cluster_sessions_df_found = cluster_sessions_df_nom_path_found.loc[~cluster_sessions_df_nom_path_found['real_cluster_session_path'].isin(pu.path_not_found_dict.values()), :]
cluster_sessions_df_found['cluster_session_rel_path'] = cluster_sessions_df_found.loc[:,'real_cluster_session_path'].str.replace(dj.config['custom']['clustering_root_data_dir'], '', regex=False)
cluster_sessions_df_found = cluster_sessions_df_found.reset_index(drop=True)
cluster_sessions_df_found

c:\users\stephan\anaconda3\envs\brody_ephys_elements_two\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,sessid,session_rat,session_userid,session_rigid,session_date,user_id,experimenter,subject_path,nominal_session_path,real_raw_session_path,raw_session_rel_path,subject_cluster_path,nominal_cluster_session_path,real_cluster_session_path,cluster_session_rel_path
0,585444,T170,zhihaol,202,2018-03-16,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T170/T170_2018_03_16,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Thomas/T170/T170_2018_03_16_600s/spikesort_202...
1,585488,T173,zhihaol,202,2018-03-16,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T173/T173_bank1_2018_03_16,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Thomas/T173/T173_2018_03_16_bank1_600s/spikeso...
2,586335,T170,zhihaol,202,2018-03-19,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T170/T170_2018_03_19,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Thomas/T170/T170_2018_03_19/spikesort_2020_09_...
3,586376,T173,zhihaol,202,2018-03-19,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T173/T173_bank0_2018_03_19,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Thomas/T173/T173_2018_03_19_bank0_600s/spikeso...
4,590577,T170,zhihaol,202,2018-04-05,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T170/T170_2018_04_05,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Thomas/T170/T170_2018_04_05/spikesort_2020_09_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,798430,T274,zhihaol,202,2020-12-13,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_13c_g0,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Thomas/T274/T274_2020_12_13c/spikesort_2020_12...
1098,798586,T274,zhihaol,202,2020-12-14,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_14_g0,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Thomas/T274/T274_2020_12_14/spikesort_2020_12_...
1099,798742,T274,zhihaol,202,2020-12-15,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,Thomas/T274/T274_2020_12_15_g0,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,//bucket.pni.princeton.edu/brody/RATTER/PhysDa...,Thomas/T274/T274_2020_12_15/spikesort_2020_12_...
1100,798897,T274,zhihaol,202,2020-12-16,zhihaol,Thomas,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...,//apps02.pni.princeton.edu/archive-labdata/bro...

# 3.  Ingest into DB (preAcquisitionSession)

## 3.1 Add/Select columns from the DF --> DB

In [84]:
#Count how many possible directories for each session we have
cluster_sessions_df_found['directory_num'] =  cluster_sessions_df_found.groupby('sessid').cumcount()
cluster_sessions_df_found = cluster_sessions_df_found.reset_index(drop=True)


cluster_sessions_df_found = cluster_sessions_df_found.rename(columns={"cluster_session_rel_path": "acquisition_post_rel_path",\
                                          "raw_session_rel_path": "acquisition_raw_rel_path"})

cluster_sessions_df_found['acquisition_type'] = 'ephys'
cluster_sessions_df_found['correct_dirs'] = 0

pre_acquisition_sessions_df = cluster_sessions_df_found[new_acquisition.PreAcquisitionSessions.heading.names]
pre_acquisition_sessions_df





,sessid,directory_num,session_rat,session_userid,session_rigid,acquisition_type,acquisition_raw_rel_path,acquisition_post_rel_path,correct_dirs
0,585444,0,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_03_16,Thomas/T170/T170_2018_03_16_600s/spikesort_202...,0
1,585488,0,T173,zhihaol,202,ephys,Thomas/T173/T173_bank1_2018_03_16,Thomas/T173/T173_2018_03_16_bank1_600s/spikeso...,0
2,586335,0,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_03_19,Thomas/T170/T170_2018_03_19/spikesort_2020_09_...,0
3,586376,0,T173,zhihaol,202,ephys,Thomas/T173/T173_bank0_2018_03_19,Thomas/T173/T173_2018_03_19_bank0_600s/spikeso...,0
4,590577,0,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_04_05,Thomas/T170/T170_2018_04_05/spikesort_2020_09_...,0
...,...,...,...,...,...,...,...,...,...
1097,798430,0,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_13c_g0,Thomas/T274/T274_2020_12_13c/spikesort_2020_12...,0
1098,798586,0,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_14_g0,Thomas/T274/T274_2020_12_14/spikesort_2020_12_...,0
1099,798742,0,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_15_g0,Thomas/T274/T274_2020_12_15/spikesort_2020_12_...,0
1100,798897,0,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_16_g0,Thomas/T274/T274_2020_12_16/spikesort_2020_12_...,0


In [85]:
so = pre_acquisition_sessions_df.groupby('directory_num').max()
pre_acquisition_sessions_df.loc[pre_acquisition_sessions_df['sessid'] == 710898, 'acquisition_post_rel_path'].values

array(['Adrian/A230/A230_bank0_2019_07_06/2019-07-06_g0_imec1/spikesort_2019_07_09_20_37_07_ks2jrc',
       'Adrian/A230/A230_bank0_2019_07_06/2019-07-06_g0_imec1/spikesort_2020_03_12_16_54_50_ks2jrc',
       'Adrian/A230/A230_bank0_2019_07_06/2019-07-06_g0_imec1/spikesort_2020_04_30_12_43_13_ks2jrc',
       'Adrian/A230/A230_bank0_2019_07_06/2019-07-06_g0_imec2',
       'Adrian/A230/2019-07-06/2019-07-06_g0_imec0',
       'Adrian/A230/2019-07-06/2019-07-06_g0_imec1/spikesort_2020_04_30_12_43_13_ks2jrc',
       'Adrian/A230/2019-07-06/2019-07-06_g0_imec2',
       'Adrian/A230/2019-07-06/2019-07-06_site_2_bank_1_g0/2019-07-06_site_2_bank_1_g0_imec0',
       'Adrian/A230/A230_bank0_2019_07_06/2019-07-06_g0_imec1/spikesort_2019_07_09_20_37_07_ks2jrc',
       'Adrian/A230/A230_bank0_2019_07_06/2019-07-06_g0_imec1/spikesort_2020_03_12_16_54_50_ks2jrc',
       'Adrian/A230/A230_bank0_2019_07_06/2019-07-06_g0_imec1/spikesort_2020_04_30_12_43_13_ks2jrc',
       'Adrian/A230/A230_bank0_2019_07_

## 3.2 Ingest to preAcquisitionSession

In [86]:
dict_keys = pre_acquisition_sessions_df.to_dict(orient='records')
for i in dict_keys:
    new_acquisition.PreAcquisitionSessions.insert1(i)

# 4. Update correct_dirs of known PreAcquisitionSessions (triggers AcquisitionSessions insert)

## 4.1  Fetch from PreAcquisitionSessions

In [4]:
preacq_sessions_df = pd.DataFrame(new_acquisition.PreAcquisitionSessions.fetch(order_by='sessid desc', as_dict=True))
idx_duplicate_raw = preacq_sessions_df['sessid'].duplicated(keep=False)
preacq_sessions_df = preacq_sessions_df[~idx_duplicate_raw]
preacq_sessions_df = preacq_sessions_df.loc[preacq_sessions_df['directory_num'] == 0, :]
preacq_sessions_df = preacq_sessions_df.reset_index(drop=True)
preacq_sessions_df

,sessid,directory_num,session_rat,session_userid,session_rigid,acquisition_type,acquisition_raw_rel_path,acquisition_post_rel_path,correct_dirs
0,798899,0,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_17_g0,Thomas/T274/T274_2020_12_17/spikesort_2020_12_...,0
1,798897,0,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_16_g0,Thomas/T274/T274_2020_12_16/spikesort_2020_12_...,0
2,798742,0,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_15_g0,Thomas/T274/T274_2020_12_15/spikesort_2020_12_...,0
3,798586,0,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_14_g0,Thomas/T274/T274_2020_12_14/spikesort_2020_12_...,0
4,798430,0,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_13c_g0,Thomas/T274/T274_2020_12_13c/spikesort_2020_12...,0
...,...,...,...,...,...,...,...,...,...
303,590577,0,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_04_05,Thomas/T170/T170_2018_04_05/spikesort_2020_09_...,0
304,586376,0,T173,zhihaol,202,ephys,Thomas/T173/T173_bank0_2018_03_19,Thomas/T173/T173_2018_03_19_bank0_600s/spikeso...,0
305,586335,0,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_03_19,Thomas/T170/T170_2018_03_19/spikesort_2020_09_...,0
306,585488,0,T173,zhihaol,202,ephys,Thomas/T173/T173_bank1_2018_03_16,Thomas/T173/T173_2018_03_16_bank1_600s/spikeso...,0


## 4.2 Find unequivocally relation between directories and session

In [10]:
idx_duplicate_raw = preacq_sessions_df['acquisition_raw_rel_path'].duplicated(keep=False)
preacq_sessions_df_unique_raw = preacq_sessions_df[~idx_duplicate_raw]
preacq_sessions_df_unique_raw = preacq_sessions_df_unique_raw.reset_index(drop=True)
idx_duplicate_post = preacq_sessions_df_unique_raw['acquisition_post_rel_path'].duplicated(keep=False)
acq_sessions_df_unique = preacq_sessions_df_unique_raw[~idx_duplicate_post]
acq_sessions_df_unique = acq_sessions_df_unique.reset_index(drop=True)
acq_sessions_df_unique

,sessid,directory_num,session_rat,session_userid,session_rigid,acquisition_type,acquisition_raw_rel_path,acquisition_post_rel_path,correct_dirs
0,798899,0,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_17_g0,Thomas/T274/T274_2020_12_17/spikesort_2020_12_...,0
1,798897,0,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_16_g0,Thomas/T274/T274_2020_12_16/spikesort_2020_12_...,0
2,798742,0,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_15_g0,Thomas/T274/T274_2020_12_15/spikesort_2020_12_...,0
3,798586,0,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_14_g0,Thomas/T274/T274_2020_12_14/spikesort_2020_12_...,0
4,798430,0,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_13c_g0,Thomas/T274/T274_2020_12_13c/spikesort_2020_12...,0
...,...,...,...,...,...,...,...,...,...
282,590577,0,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_04_05,Thomas/T170/T170_2018_04_05/spikesort_2020_09_...,0
283,586376,0,T173,zhihaol,202,ephys,Thomas/T173/T173_bank0_2018_03_19,Thomas/T173/T173_2018_03_19_bank0_600s/spikeso...,0
284,586335,0,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_03_19,Thomas/T170/T170_2018_03_19/spikesort_2020_09_...,0
285,585488,0,T173,zhihaol,202,ephys,Thomas/T173/T173_bank1_2018_03_16,Thomas/T173/T173_2018_03_16_bank1_600s/spikeso...,0


## 4.3 Update correct_dir of found session trigger ingest acquisitionSessions

In [11]:
for i in range(acq_sessions_df_unique.shape[0]):
    key = dict()
    key['sessid'] = acq_sessions_df_unique.loc[i, 'sessid']
    key['directory_num'] = 0
    key['correct_dirs'] =  1
    (new_acquisition.PreAcquisitionSessions).update1(key)

## 4.4 Check AcquisitionSessions records

In [12]:
acq_sessions_df = pd.DataFrame(new_acquisition.AcquisitionSessions.fetch(order_by='sessid desc', as_dict=True))
acq_sessions_df

,sessid,session_rat,session_userid,session_rigid,acquisition_type,acquisition_raw_rel_path,acquisition_post_rel_path
0,798899,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_17_g0,Thomas/T274/T274_2020_12_17/spikesort_2020_12_...
1,798897,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_16_g0,Thomas/T274/T274_2020_12_16/spikesort_2020_12_...
2,798742,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_15_g0,Thomas/T274/T274_2020_12_15/spikesort_2020_12_...
3,798586,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_14_g0,Thomas/T274/T274_2020_12_14/spikesort_2020_12_...
4,798430,T274,zhihaol,202,ephys,Thomas/T274/T274_2020_12_13c_g0,Thomas/T274/T274_2020_12_13c/spikesort_2020_12...
...,...,...,...,...,...,...,...
282,590577,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_04_05,Thomas/T170/T170_2018_04_05/spikesort_2020_09_...
283,586376,T173,zhihaol,202,ephys,Thomas/T173/T173_bank0_2018_03_19,Thomas/T173/T173_2018_03_19_bank0_600s/spikeso...
284,586335,T170,zhihaol,202,ephys,Thomas/T170/T170_2018_03_19,Thomas/T170/T170_2018_03_19/spikesort_2020_09_...
285,585488,T173,zhihaol,202,ephys,Thomas/T173/T173_bank1_2018_03_16,Thomas/T173/T173_2018_03_16_bank1_600s/spikeso...
